In [43]:
# 라이브러리 추출
import pandas as pd
import numpy as np
import tensorflow as tf

In [49]:
# 메서드 정의
def main(URL = 'Dataset\Regression_data.csv'):
    
    # 랜덤 시드 고정
    np.random.seed(42)
    tf.random.set_seed(42)
    
    global df, LEARNING_RATE, EPOCH_COUNT, MB_SIZE, REPORT, TRAIN_RATIO, X, y, X_train, X_test, y_train, y_test, y_pred
    
    df = load_dataset(URL) # 데이터 로드
    Regression_Model() # 회귀 모델

In [50]:
# 메서드 정의
def load_dataset(URL = 'Dataset\Regression_data.csv'):
    
    # 데이터셋 로드
    df = pd.read_csv(URL)

    # 성별 인코딩
    from sklearn.preprocessing import LabelEncoder
    encoder = LabelEncoder()
    df['Sex'] = encoder.fit_transform(df['Sex'])
    
    print(df.head())
    
    return df

In [51]:
# 매서드 정의 
def eval_accuracy():
    
	# 오차율 구하는 과정    
	# np.mean() 메서드의 이유는 미니배치 처리를 고려하여 하나의 지표로 묶어주기 위함 입니다. 
    mdiff = np.mean(np.abs((y_pred - y_test) / y_test))
    # 1 에서 오차율을 빼 정확도를 구합니다. 
    return 1 - mdiff

In [58]:
def Regression_Model():
    # 학습 모델 구현
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import r2_score
    from sklearn.metrics import mean_squared_error

    # 상수 정의
    LEARNING_RATE = 0.01
    EPOCH_COUNT = 100
    MB_SIZE = 10
    REPORT = 1
    TRAIN_RATIO = 0.8

    # 학습 데이터 분리
    X = df.drop('Rings', axis=1)
    y = df['Rings']
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=TRAIN_RATIO)

    # 모델 생성
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=128, activation='relu', input_shape=(8,)), # (8,)로 입력해야 튜플 형태로 입력되어 오류가 발생하지 않음
        tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=1)
    ])


    # 옵티마이저와 손실 함수 설정
    optimizer = tf.keras.optimizers.SGD(learning_rate=LEARNING_RATE) # SGD : 경사하강법을 기본적으로 사용하는 옵티마이저
    model.compile(loss='mean_squared_error', optimizer=optimizer) # metrics=[eval_accuracy(y_pred, y_test)] 직접 작성한 eval_accuracy를 평가지표로 사용할 수 있음

    # 학습 시작
    model.fit(X_train, y_train, epochs=EPOCH_COUNT, batch_size=MB_SIZE, verbose=REPORT)

    # 평가
    y_pred = model.predict(X_test)

    # R^2 Score 계산
    r2 = r2_score(y_test, y_pred)
    print(f'R^2 Score: {round(r2, 3)}')

    # MSE 계산
    mse = mean_squared_error(y_test, y_pred)
    print(f'Mean Squared Error: {round(mse, 3)}')


In [59]:
main()

   Sex  Length  Diameter  Height  Whole weight  Shucked weight  \
0    2   0.455     0.365   0.095        0.5140          0.2245   
1    2   0.350     0.265   0.090        0.2255          0.0995   
2    0   0.530     0.420   0.135        0.6770          0.2565   
3    2   0.440     0.365   0.125        0.5160          0.2155   
4    1   0.330     0.255   0.080        0.2050          0.0895   

   Viscera weight  Shell weight  Rings  
0          0.1010         0.150     15  
1          0.0485         0.070      7  
2          0.1415         0.210      9  
3          0.1140         0.155     10  
4          0.0395         0.055      7  
Epoch 1/100
335/335 [==============================] - 1s 2ms/step - loss: 10.0795
Epoch 2/100
335/335 [==============================] - 1s 2ms/step - loss: 7.3718
Epoch 3/100
335/335 [==============================] - 1s 2ms/step - loss: 6.4557
Epoch 4/100
335/335 [==============================] - 1s 2ms/step - loss: 6.0497
Epoch 5/100
335/335 [=======